# Equations

$ V(s_t) = \gamma ^0r(s_t) + \gamma ^1r(s_{t+1}) + \gamma ^2r(s_{t+2}) + ... + \gamma ^nr(s_{t+n})$

$ \delta (s_t) = [r(s_t) + \gamma V(s_{t+1})] - V(s_t) $

$ V(s_t) = V(s_t) + \alpha\delta $

$ \pi (s_t) = argmax_a V(s_t) $

## Temporal Difference 

In [10]:
#from plotly.graph_objs import Scatter, Layout
import matplotlib.pyplot as plt
import plotly
import numpy as np
import random
import hrr
from plotly.graph_objs import Scatter, Layout, Surface
plotly.offline.init_notebook_mode(connected=True)

In [13]:
# state 2 is where the cheese is
# 1X3 maze

def reward(s):
    if s == 0:
        return 1
    else:
        return 0
    
def valuefunc(s,V,W,bias):
    return np.dot(V[s],W)+bias
    
def policy(s,V,W,bias):
    x = random.random()
    ep = 0.01
    left = ((s-1)+len(V))%len(V)
    right = ((s+1)+len(V))%len(V)
    
    if x < ep:
            #print('Random choice')
        return random.choice([left,right])
    elif valuefunc(right,V,W,bias) == valuefunc(left,V,W,bias):
        return random.choice([left,right])
    elif valuefunc(right,V,W,bias) > valuefunc(left,V,W,bias):
        return right
    else:
        return left
        
def TD(v_size, num_ep, alpha, gamma): 
    #number of states
    nstates = v_size
    
    #length of hrr
    n = 128
    
    #encode each state as a unique hrr
    states = hrr.hrrs(n,nstates)
    
    #initialize weight vector with small random values
    W = np.random.random(n)*.01
    bias = 0
    
    num_episodes = num_ep
    #V = [0]*v_size
    #V[9] = -10 # sudo barrier
    #print('Initial V is',V)
    #plt.plot([x for x in range(v_size)], V, 'bo',[x for x in range(v_size)], V, 'k')
    #plt.axis([0, v_size, 0, 1])
    #plt.show()
    for x in range(num_episodes):
        #print('Episode:',x)
        s = random.randrange(0,v_size)
        t_state = 0
        current_value = 0
        next_value = 0
        while s != t_state:
            new_state = policy(s,states,W,bias) #get new state
            r = reward(s) #get reward at state s
            
            ## Get the learned value V[s] of the current state
            ## Note this is a simple perceptron, or single-layer
            ## neural network which produces a weighted sum of the
            ## HRR for the current state.
            current_value = np.dot(states[s],W)+bias
            next_value = np.dot(states[new_state],W)+bias
            #print("Value of V[",s,"] is",current_value)
            #print("Value of V[",new_state,"] is",next_value)
            #error = (r + gamma*V[new_state]) - V[s] #calc error
            error = (r + gamma*next_value)-current_value
            
            #V[s] += alpha*error #update V
            W += alpha*error*states[s]
            #bias += alpha*error
            s = new_state #update state s
           
        #reward absorbed
        r = reward(s)
        #print("final state is",s)
        #error = r - V[s]
        current_value = np.dot(states[s],W)+bias
        error = r - current_value
        #print("Error =",error)
        W += alpha*error*states[s]
        #bias += alpha*error
        #print(list(map(lambda x: np.dot(x,W)+bias, states)))
        #V[s] += error
        ##############
        #print(V)
    V = list(map(lambda x: np.dot(x,W)+bias, states))
    plotly.offline.iplot([
    dict(x=[x for x in range(len(V))] , y=V, type='scatter')
    ])
    '''
    print('Final V is',V)
    plt.plot([x for x in range(v_size)], V, 'bo',[x for x in range(v_size)], V, 'k')
    plt.axis([0, v_size, 0, 1])
    plt.show()'''

In [17]:
TD(10,10000,.01,.5)